In [1]:
import sys
sys.path.append("..")

In [2]:
from src.datamodules import SN7DataModule
import pandas as pd
from argparse import Namespace
from src.datasources import S2_BANDS
import torch
from src.FSRCNN import FSRCNNModule

In [3]:
date_range = pd.date_range(start=f"2019-12-30", end=f"2020-01-31")
dm = SN7DataModule(date_range=date_range,s2_bands=S2_BANDS['true_color'],
                   only_whole_scenes=True,
                   batch_size=1, collate_2toN=True, normalize=True, standardize_sentinel=False)
dm.setup()

In [4]:
modelname = "FSRCNN_SSIM"
net_args = Namespace(in_channels=3, out_channels=3, upscale_factor=2, additional_scaling=None)
model = FSRCNNModule.load_from_checkpoint("gs://fdl_srhallucinate/models/sisr_FSRCNN_SSIM-epoch=7-step=7503.ckpt", net_args=net_args,
                                          shiftnet=True)

# model = FSRCNNModule(net_args=net_args, shiftnet=True)
# model = FSRCNNModule.load_from_checkpoint(PATH)
model.eval()

FSRCNNModule(
  (net): FSRCNN(
    (features): Sequential(
      (0): Conv2d(3, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
      (1): PReLU(num_parameters=1)
    )
    (shrink): Sequential(
      (0): Conv2d(64, 12, kernel_size=(1, 1), stride=(1, 1))
      (1): PReLU(num_parameters=1)
    )
    (map): Sequential(
      (0): Conv2d(12, 12, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): Conv2d(12, 12, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (2): Conv2d(12, 12, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (3): Conv2d(12, 12, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (4): PReLU(num_parameters=1)
    )
    (expand): Sequential(
      (0): Conv2d(12, 64, kernel_size=(1, 1), stride=(1, 1))
      (1): PReLU(num_parameters=1)
    )
    (deconv): ConvTranspose2d(64, 3, kernel_size=(9, 9), stride=(2, 2), padding=(4, 4), output_padding=(1, 1))
    (sigmoid): Sigmoid()
  )
  (registration_model): ShiftNet(
    (layer1): Sequenti

In [5]:
# TODO this SRGAN weights fail bc of renaming of generator
# from src.SRGAN import SRResNetModule

# modelname = "FSRCNN_SRResNet"
# net_args = Namespace(in_channels=3, out_channels=3, upscale_factor=2, additional_scaling=None)
# model = SRResNetModule.load_from_checkpoint("gs://fdl_srhallucinate/models/sisr_SRResNet_SSIM-epoch=5-step=5627.ckpt", 
#                                             net_args=net_args,
#                                             shiftnet=True)

# # model = FSRCNNModule(net_args=net_args, shiftnet=True)
# # model = FSRCNNModule.load_from_checkpoint(PATH)
# model.eval()

SRResNetModule(
  (net): Generator(
    (block1): Sequential(
      (0): Conv2d(3, 64, kernel_size=(9, 9), stride=(1, 1), padding=(4, 4))
      (1): PReLU(num_parameters=1)
    )
    (block2): ResidualBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (prelu): PReLU(num_parameters=1)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (block3): ResidualBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (prelu): PReLU(num_parameters=1)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_run

In [5]:
# from pytorch_lightning.utilities.cloud_io import load as pl_load


# # Load directly from the bucket works if installed gcsfs
# checkpoint = pl_load("gs://fdl_srhallucinate/models/sisr_FSRCNN_SSIM-epoch=7-step=7503.ckpt", 
#                      map_location=lambda storage, loc: storage)

# model.load_state_dict(checkpoint['state_dict'], strict=True)

<All keys matched successfully>

In [5]:
model.device

device(type='cpu')

In [10]:
from src import pred_scenes

scenes_preds_list = pred_scenes.predictions_to_bucket(model,name_prediction_folder=modelname,data_module=dm,verbose=True)
print("Done")


Copying predictions to: gs://fdl_srhallucinate/spacenet/predictions/FSRCNN_SSIM
Dataloader val (0/5) making predictions for scene L15-0368E-1245N_1474_3210_13
	 Input shape: torch.Size([1, 8, 3, 490, 490]) output shape: torch.Size([1, 3, 980, 980]) highres shape: torch.Size([1, 1, 3, 1024, 1024])
Dataloader val (1/5) making predictions for scene L15-0632E-0892N_2528_4620_13
	 Input shape: torch.Size([1, 8, 3, 490, 489]) output shape: torch.Size([1, 3, 980, 978]) highres shape: torch.Size([1, 1, 3, 1024, 1023])
Dataloader val (2/5) making predictions for scene L15-1049E-1370N_4196_2710_13
	 Input shape: torch.Size([1, 8, 3, 490, 490]) output shape: torch.Size([1, 3, 980, 980]) highres shape: torch.Size([1, 1, 3, 1024, 1024])
Dataloader val (3/5) making predictions for scene L15-1210E-1025N_4840_4088_13
	 Input shape: torch.Size([1, 8, 3, 490, 490]) output shape: torch.Size([1, 3, 980, 980]) highres shape: torch.Size([1, 1, 3, 1024, 1024])
Dataloader val (4/5) making predictions for scen

In [11]:
# Create manually the name_prediction_folder in you GEE assets 
# pred_scenes.create_image_collection_gee(scenes_preds_list,name_prediction_folder=modelname,gee_user="gonzmg88")

Creating Image Collection in the GEE
To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&state=TcgAprbO43RPbQXaARUNdsbdV42oxO&prompt=consent&access_type=offline

The authorization workflow will generate a code, which you should paste in the box below. 




Enter the authorization code: 
 4/1AY0e-g6IQGX5TwuwLJPijTC1KZtXLHq_jGy1nM60VNHfEUaktCvEdxiWMnQ


Creating Image Collection in GEE...: 100%|██████████| 46/46 [01:18<00:00,  1.71s/it]


## Uploading S2 images and Planet Images

Info taken from:
https://developers.google.com/earth-engine/Earth_Engine_asset_from_cloud_geotiff

In [7]:
import json
with open("../src/scene_split.json","r") as fh:
    train_val_test_split = json.load(fh)
train_val_test_split = train_val_test_split["split"]

'FSRCNN_SSIM'

In [ ]:
request = {
  'type': 'IMAGE',
  'gcs_location': {
    'uris': ['gs://ee-docs-demos/COG_demo.tif']
  },
  'properties': {
    'split': 'train'
  },
  'startTime': '2016-01-01T00:00:00.000000000Z',
  'endTime': '2016-12-31T15:01:23.000000000Z',
}

# Where Earth Engine assets are kept.
url = 'https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/assets?assetId={}'

In [ ]:
files_s2 = !gsutil ls gs://fdl_srhallucinate/spacenet/train/*/S2L2A/*.tif
len(files_s2)

In [ ]:
import os
from tqdm import tqdm
import rasterio
import numpy as np

# Your user folder name and new asset name.
asset_collection_name = f'users/{gee_username}/Spacenet7_S2'
# TODO add cleareance score to add as a property

for s2f in tqdm(files_s2):
    date_s2 = os.path.splitext(os.path.basename(s2f))[0]
    scene_id = os.path.basename(os.path.dirname(os.path.dirname(s2f)))
    with rasterio.open(s2f) as rst:
        cloud_mask = rst.read(13)
    
    clearance_mask = ((cloud_mask != 9) & (cloud_mask > 1))
    clearance = float(np.mean(clearance_mask))
    
    start_time =   f'{date_s2}T00:00:00.000000000Z'
    end_time = f'{date_s2}T23:59:00.000000000Z'
    request_copy = request.copy()
    request_copy['gcs_location']['uris'] = [s2f]
    request_copy['startTime'] = start_time
    request_copy['endTime'] = end_time
    request_copy["properties"] = {"scene_id": scene_id, 
                                  "split": train_val_test_split[scene_id],
                                  "clearance": clearance}
    
    asset_name = f"{scene_id}-{date_s2}"
    asset_id = f"{asset_collection_name}/{asset_name}"
    response = session.post(
        url = url.format(asset_id),
        data = json.dumps(request_copy))

    assert response.status_code == 200, f'{json.loads(response.content)}'
    

In [ ]:
files_planet = !gsutil ls gs://fdl_srhallucinate/spacenet/train/*/imagesCOG/global_monthly_2020_01*.tif
len(files_planet)

In [ ]:
import re

# Your user folder name and new asset name.
asset_collection_name = f'users/{gee_username}/Spacenet7Planet'

for fpf in tqdm(files_planet):
    basename_pred = os.path.splitext(os.path.basename(fpf))[0]
    year, month= re.match("global_monthly_(\d{4})_(\d{2})_mosaic_", basename_pred).groups()
    
    scene_id = os.path.basename(os.path.dirname(os.path.dirname(fpf)))
    start_time =   f'{year}-{month}-01T00:00:00.000000000Z'
    end_time = f'{year}-{month}-31T23:59:00.000000000Z'
    request_copy = request.copy()
    request_copy['gcs_location']['uris'] = [fpf]
    request_copy['startTime'] = start_time
    request_copy['endTime'] = end_time
    request_copy["properties"] = {"scene_id": scene_id, "split": train_val_test_split[scene_id]}
    
    asset_name = f"{scene_id}-{year}-{month}"
    asset_id = f"{asset_collection_name}/{asset_name}"
    
    response = session.post(
        url = url.format(asset_id),
        data = json.dumps(request_copy))

    assert response.status_code == 200, f'{json.loads(response.content)}'